In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('default')

from z3 import *

import sys
sys.path.append("../common")

from utils import visualize, read_instance, PositionedRectangle

from SAT import SAT_solve, SAT_optimize

#reloads external modules when they are changed
%load_ext autoreload
%autoreload 2

In [ ]:
ins_filename = "../instances/ins-36.txt"
# ins_filename = "ins-test.txt"
W, n, rectangles = read_instance(ins_filename)

try:
	H, positioned_rectangles = SAT_optimize(W, rectangles, verbose=True)
	plot = True
except Z3Exception:
	plot = False
	print("\n----Timeout----\n")

In [ ]:
if plot:
	fig, ax = plt.subplots(figsize = (10, 15), dpi = 100)

	fig, ax = visualize(W, H, positioned_rectangles, ax = ax)
	fig.suptitle(f"{ins_filename.split('.')[0]}, {n} rectangles, W = {W}, H = {H}")
	fig.tight_layout(pad = 1)
	#plt.savefig("../" + ins_filename.split('.')[0])
	plt.show()